In [1]:
import pandas as pd # org orra t
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr 
import datetime 
import pymongo as md
from pymongo import MongoClient
import pprint
from bson.objectid import ObjectId
import plotly.graph_objects as go 
import plotly.express as px
from scipy.stats import norm
from matplotlib import pyplot as plt

C:\Users\Sophia\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
#busca ativos do Índice Ibovespa
c = 0
p = 18
acoesb3 = pd.DataFrame(columns=["empresa","ticker","indice"])
while c<p:
    page = rq.get('http://bvmf.bmfbovespa.com.br/indices/AcoesIndice.aspx?idioma=pt-BR&pg='+str(c))# somente as do indice
    soup = bs(page.content, 'html.parser')
    table = soup.find('div', class_='large-12 columns')
    lista = table.find_all('td')
    i = 0
    while i < len(lista)-3:
        acoesb3 = acoesb3.append({"empresa": ' '.join((lista[i].text.split())),
                                  "ticker": ''.join(lista[i+1].text.split()),
                                  "indice": ' '.join(lista[i+2].text.split())}, ignore_index=True)
        i+=3
    c+=1
 #retirando fundos de investimento   
acoes = acoesb3.query("indice != 'IFIX'")#retirando fundos de investimentos
acoes1 = acoes.drop_duplicates()
#incluindo índice Ibovespa
acoes1 = acoes1.append({"empresa":'IBOVESPA',
                       "ticker":'^BVSP',
                       "indice":'IBOV'}, ignore_index=True)

In [103]:
#conectando ao mongo
con = MongoClient()
db = con.price
#obs:No MongoDB, a base só é criada se receber conteúdo 

In [ ]:
#consultando precos histórico
for i in acoes1['ticker']:
    if i == 'KEPL3' or i =='KMPR34' or i == 'JSLG3' or i== 'JPMC34' or i =='LINX3' or i =='K1EY34': #ações com falta de preços
        continue
    if i in db.list_collection_names():
        continue
    else:
        coll = db[i]
        if i =='^BVSP':
            price = pdr.DataReader(str(i), 'yahoo',start=datetime.datetime(2010, 1, 4), end=datetime.datetime(2020, 5, 29)) 
        else:
            price = pdr.DataReader(str(i+'.SA'), 'yahoo',start=datetime.datetime(2010, 1, 4), end=datetime.datetime(2020, 5, 29))
        price = price.reset_index()
        data_dict = price.to_dict("records")
        #coll.insert_one({"index":i,"data":data_dict})
        coll.insert_many(data_dict)
        n = coll.count()
    print(i, n)  

In [154]:
#consulta lista de collections
for i in acoes1.ticker:
    docs = db.i.find({},{"_id": 0, "Low":0, "High":0, "Open":0, "Volume":0, "Adj Close":0})
    d = get_collection(i)
        len(lc)

In [148]:
cursor = db.list_collection_names()
for c in cursor:
    #print(list(cursor))
    docs = db.c.find({},{"_id": 0, "Low":0, "High":0, "Open":0, "Volume":0, "Adj Close":0})
    for dc in docs:
        print(dc)
    #bp = pd.DataFrame(list(docs))
    #print("Medidas de Tendências Central", d )
    #medida_central(bp.Close)

In [94]:
#Análise Descritiva 
#Medidas de Tendência Central
def medida_central(col):
    #+print("Medidas de Tendência Central") 
    #print(Nomes[i])
    print('Média:', round(col.mean(),2 ))
    print('Mediana:',round(col.median(),2))
    print('Moda:', round(col.mode().iloc[0]),2)
    print('Ponto Médio:',round((max(col) - min(col))/2,2))
    print("\n")   
    
def medida_dispersao(col):
    #print("Medidas de Dispersão")
    #print(Nomes[i])
    print('Amplitude:',(round(max(col) - min(col),2)))
    d1 = round(col.std(),2)
    print('Desvio Padrão σ:',d1)
    print('Variância:', round(col.var(),2))
    print('Coeficiente de variação', round(col.std()/col.median(),2))
    print('Dispersão em relação aos Desvios Padrão')
    print('Itens a 1 σ ', round(col[col[col >((col.mean()+d1)*(-1))]<((col.mean()+d1))].count()/col.count()*100,2),'%')
    print('Itens a 2 σ ', round(col[col[col >((col.mean()+d1)*(-2))]<((col.mean()+d1)*2)].count()/col.count()*100,2),'%')
    print('Itens a 3 σ ', round(col[col[col >((col.mean()+d1)*(-3))]<((col.mean()+d1)*3)].count()/col.count()*100,2),'%')
    print('\n')  
    
    
def distr_freq(col,classe):
    print('Classe','Limite_Inf','Pto_Médio','Limite_Sup','Fronteira','Freq_Absoluta','Freq_Relativa',
          'Freq_Acumulada','Freq_Acumulada_%')
    inter = (max(col) - min(col))//classe
    tot = col.count()
    menor = int(min(col)//1)
    #lista com intervalo da frequencia
    c=ac=ct= 0
    while menor < max(col):
        li = menor #limit inf
        ls = menor+inter #limit sup
        for i in col:
            if  li-0.5 < i <= ls-0.5:
                ct = ct + 1
        ac = ac + ct
        #frq = pd.concat([frq.append,(c,li, ((li+ls)/2),ls, (ls-1+0.5),ct, round(ct/tot,2), ac, round(ac/tot,2))], axis=0)
        print(c,"\t" ,li//1,"\t", ((li+ls)/2),"\t",ls, "\t",(ls-1+0.5),"\t",ct, "\t", round(ct/tot,2),"\t", ac,"\t", round(ac/tot,2))
        c = c + 1
        menor = menor + inter
    print("\n")
    #return (frq)
    
def medidas_associacao(col,col2,i,i2):
    #print("Medidas de Associação")
    print("A correlação  entre as ações")
    print( i.replace(".SA",""), "/", i2.replace(".SA",""), "é",round(col.corr(col2),2))  #np.corrcoef
    print("\nE a Covariância")
    print(np.cov(col,col2) )
    print("\n") 